In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
# install tf 2.0
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
# tf.compat.v1.disable_v2_behavior()

print(tf.__version__)
import requests
import pandas as pd 

2.0.0-alpha0


In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
pd.options.display.max_rows = 700
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth', 250)
pd.set_option('expand_frame_repr', True)
pd.get_option("display.max_rows")

700

In [0]:
import os
from glob import glob

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

SEED = 42


def _float_list_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _int64_list_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def create_generator_for_ffn(
        file_list,
        mode='train'):

    # file_list = glob(os.path.join(data_dir, '*.csv'))

    for full_file_path in file_list:
        # full_file_path = os.path.join(data_dir, file_name)
        if not os.path.exists(full_file_path):
            raise FileNotFoundError("File %s not found" % full_file_path)
        df = pd.read_csv(full_file_path, encoding='utf8')

        # so train test split
        if mode == 'train':
            df, _ = train_test_split(df, test_size=0.2, random_state=SEED)
        else:
            _, df = train_test_split(df, test_size=0.2, random_state=SEED)

        for _, row in df.iterrows():
            q_vectors = np.fromstring(row.question_bert.replace(
                '[[', '').replace(']]', ''), sep=' ')
            a_vectors = np.fromstring(row.answer_bert.replace(
                '[[', '').replace(']]', ''), sep=' ')
            vectors = np.stack([q_vectors, a_vectors], axis=0)
            if mode in ['train', 'eval']:
                print(vectors)                                      #added 
                yield vectors, 1
            else:
                print(vectors)                                        #added 
                yield vectors


def ffn_serialize_fn(features):
    features_tuple = {'features': _float_list_feature(
        features[0].flatten()), 'labels': _int64_feature(features[1])}
    example_proto = tf.train.Example(
        features=tf.train.Features(feature=features_tuple))
    return example_proto.SerializeToString()


def make_tfrecord(data_dir, generator_fn, serialize_fn, suffix='', **kwargs):
    """Function to make TF Records from csv files
    This function will take all csv files in data_dir, convert them
    to tf example and write to *_{suffix}_train/eval.tfrecord to data_dir.

    Arguments:
        data_dir {str} -- dir that has csv files and store tf record
        generator_fn {fn} -- A function that takes a list of filepath and yield the
        parsed recored from file.
        serialize_fn {fn} -- A function that takes output of generator fn and convert to tf example

    Keyword Arguments:
        suffix {str} -- suffix to add to tf record files (default: {''})
    """
    file_list = glob(os.path.join(data_dir, '*.csv'))
    train_tf_record_file_list = [
        f.replace('.csv', '_{0}_train.tfrecord'.format(suffix)) for f in file_list]
    test_tf_record_file_list = [
        f.replace('.csv', '_{0}_eval.tfrecord'.format(suffix)) for f in file_list]
    for full_file_path, train_tf_record_file_path, test_tf_record_file_path in zip(file_list, train_tf_record_file_list, test_tf_record_file_list):
        print('Converting file {0} to TF Record'.format(full_file_path))
        with tf.io.TFRecordWriter(train_tf_record_file_path) as writer:
            for features in generator_fn([full_file_path], mode='train', **kwargs):
                example = serialize_fn(features)
                writer.write(example)
        with tf.io.TFRecordWriter(test_tf_record_file_path) as writer:
            for features in generator_fn([full_file_path], mode='eval', **kwargs):
                example = serialize_fn(features)
                writer.write(example)


def create_dataset_for_ffn(
        data_dir,
        mode='train',
        hidden_size=768,
        shuffle_buffer=10000,
        prefetch=10000,
        batch_size=32):

    tfrecord_file_list = glob(os.path.join(
        data_dir, '*_FFN_{0}.tfrecord'.format((mode))))
    if not tfrecord_file_list:
        print('TF Record not found')
        make_tfrecord(
            data_dir, create_generator_for_ffn,
            ffn_serialize_fn, 'FFN')

    dataset = tf.data.TFRecordDataset(tfrecord_file_list)

    def _parse_ffn_example(example_proto):
        feature_description = {
            'features': tf.io.FixedLenFeature([2*768], tf.float32),
            'labels': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        }
        feature_dict = tf.io.parse_single_example(
            example_proto, feature_description)
        return tf.reshape(feature_dict['features'], (2, 768)), feature_dict['labels']
    dataset = dataset.map(_parse_ffn_example)

    dataset = dataset.shuffle(shuffle_buffer)

    dataset = dataset.prefetch(prefetch)

    dataset = dataset.batch(batch_size)
    return dataset

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

import tensorflow as tf
import tensorflow.keras.backend as K


class FFN(tf.keras.layers.Layer):
    def __init__(
            self,
            hidden_size=768,                                                                #SG edit from 768 4-24-19
            dropout=0.2,
            residual=True,
            name='FFN',
            **kwargs):
        """Simple Dense wrapped with various layers
        """

        super(FFN, self).__init__(name=name, **kwargs)
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.residual = residual
        self.ffn_layer = tf.keras.layers.Dense(
            units=hidden_size,
            use_bias=True
        )

    def call(self, inputs):
        ffn_embedding = self.ffn_layer(inputs)
        ffn_embedding = tf.keras.layers.ReLU()(ffn_embedding)
        if self.dropout > 0:
            ffn_embedding = tf.keras.layers.Dropout(
                self.dropout)(ffn_embedding)
#         ffn_embedding = self.ffn_layer(inputs)  #SG edit from 768 4-24-19
#         ffn_embedding = tf.keras.layers.ReLU()(ffn_embedding)  #SG edit from 768 4-24-19
#         if self.dropout > 0:  #SG edit from 768 4-24-19
#             ffn_embedding = tf.keras.layers.Dropout(  #SG edit from 768 4-24-19
#                 self.dropout)(ffn_embedding)  #SG edit from 768 4-24-19


        if self.residual:
            ffn_embedding += inputs
        return ffn_embedding


class MedicalQAModel(tf.keras.Model):
    def __init__(self, name=''):
        super(MedicalQAModel, self).__init__(name=name)
        self.q_ffn = FFN(name='QFFN', input_shape=(768,))
        self.a_ffn = FFN(name='AFFN', input_shape=(768,))

    def call(self, inputs):
#         print(inputs)
#         tf.print(inputs)
        q_bert_embedding, a_bert_embedding = tf.unstack(inputs, axis=1)
#         print(q_bert_embedding)
#         print(a_bert_embedding)
#         tf.print(q_bert_embedding)
#         tf.print(a_bert_embedding)
        q_embedding, a_embedding = self.q_ffn(
            q_bert_embedding), self.a_ffn(a_bert_embedding)
        return tf.stack([q_embedding, a_embedding], axis=1)


class BioBert(tf.keras.Model):
    def __init__(self, name=''):
        super(BioBert, self).__init__(name=name)

    def call(self, inputs):

        # inputs is dict with input features
        input_ids, input_masks, segment_ids = inputs
        # pass to bert
        # with shape of (batch_size/2*batch_size, max_seq_len, hidden_size)
        # TODO(Alex): Add true bert model
        # Input: input_ids, input_masks, segment_ids all with shape (None, max_seq_len)
        # Output: a tensor with shape (None, max_seq_len, hidden_size)
        fake_bert_output = tf.expand_dims(tf.ones_like(
            input_ids, dtype=tf.float32), axis=-1)*tf.ones([1, 1, 768], dtype=tf.float32)
        max_seq_length = tf.shape(fake_bert_output)[-2]
        hidden_size = tf.shape(fake_bert_output)[-1]

        bert_output = tf.reshape(
            fake_bert_output, (-1, 2, max_seq_length, hidden_size))
        return bert_output


class MedicalQAModelwithBert(tf.keras.Model):
    def __init__(
            self,
            hidden_size=768,
            dropout=0.2,
            residual=True,
            activation=tf.keras.layers.ReLU(),
            name=''):
        super(MedicalQAModelwithBert, self).__init__(name=name)
        self.biobert = BioBert()
        self.q_ffn_layer = FFN(
            hidden_size=hidden_size,
            dropout=dropout,
            residual=residual,
            activation=activation)
        self.a_ffn_layer = FFN(
            hidden_size=hidden_size,
            dropout=dropout,
            residual=residual,
            activation=activation)

    def _avg_across_token(self, tensor):
        if tensor is not None:
            tensor = tf.reduce_mean(tensor, axis=1)
        return tensor

    def call(self, inputs):

        q_bert_embedding, a_bert_embedding = self.biobert(inputs)

        # according to USE, the DAN network average embedding across tokens
        q_bert_embedding = self._avg_across_token(q_bert_embedding)
        a_bert_embedding = self._avg_across_token(a_bert_embedding)

        q_embedding = self.q_ffn_layer(q_bert_embedding)
        a_embedding = self.a_ffn_layer(a_bert_embedding)

        return tf.stack([q_embedding, a_embedding], axis=1)

      
      
# def qa_pair_cross_entropy_loss(y_true, y_pred):
#     y_true = tf.eye(tf.shape(y_pred)[0])
#     q_embedding, a_embedding = tf.unstack(y_pred, axis=1)
#     similarity_matrix = tf.matmul(
#         q_embedding, a_embedding, transpose_b=True)
#     similarity_matrix_logits = tf.math.sigmoid(similarity_matrix)
#     return tf.keras.losses.categorical_crossentropy(y_true, similarity_matrix_logits, from_logits=True)

def qa_pair_cross_entropy_loss(y_true, y_pred):
    y_true = tf.eye(tf.shape(y_pred)[0])
    q_embedding, a_embedding = tf.unstack(y_pred, axis=1)
    similarity_matrix = tf.matmul(
        a = q_embedding, b = a_embedding, transpose_b=True)
    similarity_matrix_softmaxed = tf.nn.softmax(similarity_matrix)
    K.print_tensor(similarity_matrix_softmaxed, message="similarity_matrix_softmaxed is: ")
    return tf.keras.losses.categorical_crossentropy(y_true, similarity_matrix_softmaxed, from_logits=False)



In [0]:
# training config
batch_size = 64
num_epochs=35
learning_rate=0.0001
validation_split=0.2
shuffle_buffer=50000
prefetch=50000
data_path='/content/gdrive/My Drive/mqa_tf_record'
model_path = '/content/gdrive/My Drive/mqa_models/ffn_model_cross_entropy'

In [0]:
  d = create_dataset_for_ffn(
      data_path, batch_size=batch_size, shuffle_buffer=shuffle_buffer, prefetch=prefetch)
  eval_d = create_dataset_for_ffn(
      data_path, batch_size=batch_size, mode='eval')
  medical_qa_model = MedicalQAModel()
  optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
  medical_qa_model.compile(
      optimizer=optimizer, loss=qa_pair_cross_entropy_loss)

  epochs = num_epochs
  loss_metric = tf.keras.metrics.Mean()

#   history = medical_qa_model.fit(d, epochs=epochs, validation_data=eval_d)
#   history = medical_qa_model.train_on_batch(d)

In [0]:
model_path2 = '/content/gdrive/My Drive/mqa_models/ffn_model_cross_entropy.ckpt'

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path2, monitor='loss', verbose=1, save_best_only=True)

In [0]:
checkpoint_dir = os.path.dirname(model_path2)
print(checkpoint_dir)

/content/gdrive/My Drive/mqa_models


In [0]:
medical_qa_model.load_weights(model_path2)

In [0]:
history = medical_qa_model.fit(d, epochs=80, validation_data=eval_d )

Epoch 1/80


W0427 05:23:58.006298 140313095235456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:2924: Print (from tensorflow.python.ops.logging_ops) is deprecated and will be removed after 2018-08-20.
Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to use tf.print in python 2.7, users must make sure to import
the following:

 

    253/Unknown - 14s 55ms/step - loss: 1.3278

KeyboardInterrupt: ignored

In [0]:
file_id = '1anbEwfViu9Rzu7tWKgPb_We1EwbA4x1-'

download_file_from_google_drive(file_id, 'eHealthForum.csv')

In [0]:
eHealthPD=pd.read_csv( 'eHealthForum.csv')

In [0]:
askDocspd.head()

,question,answer,question_bert,answer_bert
0,Trying to get y friend to see a doctor but she wants to wait and see what happens. I'm worried sick and would like to know hat she might haveMy friend found a dime sized red lump a bit to the right of her clitoris in the shower this morning. its ...,This is most likely a Bartholin's Gland Cyst. On either side of the labia majora is a called a Bartholin's gland. Occasionally the ducts that empty the gland can become blocked causing the gland to develop a cyst. Sometimes they can go away on th...,[[ 5.03139086e-02 1.75226659e-01 -1.97714433e-01 -7.32559934e-02\n 3.20691824e-01 -1.78071290e-01 2.48940829e-02 -3.78650844e-01\n -3.47410381e-01 2.75857508e-01 -2.56084532e-01 2.22180486e-01\n -1.10391743e-01 1.29870668e-01 1.20820425...,[[-9.86990631e-02 2.09356025e-01 -4.21857655e-01 -3.36655468e-01\n 4.87976819e-01 -7.02803209e-03 -3.16477306e-02 -1.65782422e-01\n -2.62371778e-01 2.40044087e-01 -2.24384233e-01 1.41379043e-01\n 1.54396653e-01 -2.18005907e-02 1.46356851...
1,"Ease my worries pleaseMy husband is about to see a hematologist and googling around has me worried that he's got something serious, like aplastic anemia. I know we'll find out soon enough, but in the meantime, could you please suggest alternative...","Would be helpful with the exact values of the CBC. And B12, Folate and Ferritin values.\n\nAre liver and kidney function tests okay?\n\nWhat are his ethnic origins?,Is he diabetic. Does he take any other medication?,Does he take aspirin?\nDid he ...",[[-6.82685152e-02 4.29603830e-02 -3.32764566e-01 -6.10484481e-02\n 1.12031415e-01 -2.02944130e-01 -9.14147869e-02 -2.58494318e-01\n -2.18344301e-01 2.87112653e-01 -6.67973831e-02 -9.51428413e-02\n -8.95148739e-02 -1.64768383e-01 2.71721900...,[[ 2.15814300e-02 -8.26028138e-02 -1.79856256e-01 -1.90481871e-01\n 2.28367019e-02 -9.73006934e-02 -1.17688790e-01 -1.57697827e-01\n -1.08430490e-01 1.52954862e-01 1.85464919e-01 -1.62801355e-01\n 2.00268812e-02 -2.12160777e-02 4.19895537...
2,"Severe shoulder, neck and back pain after tetanus vaccinationA little less than a week ago I received my tetanus booster shot (the one you receive every decade, at least here in Australia).\n\nA few days later I began experiencing pain around my ...","I seriously don't think your pain derives from the vaccination.\n\nIt looks like trapezoid contracture (is it the right word in english?).\n\nHeat and posture care should help you, elseways some muscular relaxant could be helpful.\n\nAnyways, at ...",[[-9.59900767e-02 3.72468904e-02 -3.78887504e-01 -4.50596698e-02\n 1.33349165e-01 -4.34329249e-02 4.04662602e-02 -2.44166061e-01\n -1.02068119e-01 4.49610293e-01 -5.45308404e-02 -8.46140832e-02\n -4.96546812e-02 -2.82016784e-01 3.25671524...,[[-1.67309746e-01 6.44435063e-02 9.25731473e-03 1.30308941e-01\n 1.98665693e-01 1.11179024e-01 3.86667326e-02 -2.81090587e-01\n -1.66967526e-01 2.28748009e-01 6.09516762e-02 2.41061822e-02\n 2.80033629e-02 -1.44916847e-01 2.37973034...
3,"Knee problemHere's the run down. This past weekend my wife and I took part in a 3-day walk for breast cancer. It was 60 miles, so clearly a lot of work on my knees. Day 1 went fine, feet hurt, but to be expected. Day two came and I was having...",It's possible that you caused some damage while playing sports and now with a heavy amount of activity caused aggravation of that injury. Its possible to cause a small tear in a meniscus and go years without any problems until a specific event ca...,[[-1.29183223e-02 -2.05045454e-02 -5.12924075e-01 -1.66846707e-01\n 7.68833160e-02 -2.10388333e-01 -1.76014677e-01 -1.63012460e-01\n -2.01517850e-01 4.29517597e-01 -5.23534641e-02 1.08745629e-02\n -2.42132619e-02 2.21630232e-03 2.81971633...,[[-6.90914094e-02 -1.75325274e-01 -2.17794701e-01 -3.16970319e-01\n 4.53783050e-02 2.69050837e-01 -8.84563774e-02 -5.35140410e-02\n -1.40716955e-01 2.62832701e-01 5.61185367e-02 -7.53825307e-02\n -3.30312178e-03 -1.45833969e-01 7.09648505...
4,Had a strange

In [0]:
def QgetEmbeds(x):
    qt = x['question_bert']
    at = x['answer_bert']
    qtn = np.fromstring(
                               qt.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' ')
    atn = np.fromstring(
                               at.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' ')
    textinput = tf.constant([qtn, atn])
    textinputF = tf.dtypes.cast(textinput , dtype=tf.float32) 
    textinputE = tf.expand_dims(textinputF, 0)
#     trythis = medical_qa_model(textinputE)
    q_embedding, a_embedding = tf.unstack(medical_qa_model(textinputE), axis=1)
    return q_embedding.numpy()

In [0]:
askDocspd['Q_FFNN_embeds'] = askDocspd.apply(QgetEmbeds, axis=1)

In [0]:
askDocspd['Q_FFNN_embeds'].head()

0    [[0.05031391, 0.17522666, 0.71465725, 0.5769979, 0.32069182, -0.17807129, 1.2392216, -0.37865084, 0.1199767, 0.2758575, -0.002445519, 0.3054453, -0.11039174, 0.56973076, 0.120820425, -0.107291855, -0.17624098, 0.23657487, -0.23213816, 0.33558795,...
1    [[-0.068268515, 0.042960383, 0.17118096, 0.26986143, 0.112031415, -0.20294413, 0.7771517, -0.25849432, 0.59284496, 0.6635877, 0.3940065, -0.09514284, -0.089514874, 0.33704883, 0.2717219, -0.18564875, 0.1500851, 1.5225036, -0.21757159, 0.2978152, ...
2    [[-0.09599008, 0.03724689, 0.10206425, 0.4240935, 0.13334917, -0.043432925, 1.1233978, -0.24416606, 0.57163686, 0.4496103, -0.05453084, -0.060560953, -0.04965468, -0.28201678, 0.32567152, -0.12495111, -0.051384494, 1.0094256, -0.15498431, 0.43008...
3    [[-0.012918322, -0.020504545, 0.4435106, 0.19794388, 0.076883316, -0.21038833, 0.5508412, -0.16301246, 0.53401446, 0.48718905, 0.5478626, 0.13116859, -0.024213262, 0.0022163023, 0.28197163, -0.17161314, 0.30555886, 0.1917058, -0.4

In [0]:
def AgetEmbeds(x):
    qt = x['question_bert']
    at = x['answer_bert']
    qtn = np.fromstring(
                               qt.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' ')
    atn = np.fromstring(
                               at.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' ')
    textinput = tf.constant([qtn, atn])
    textinputF = tf.dtypes.cast(textinput , dtype=tf.float32) 
    textinputE = tf.expand_dims(textinputF, 0)
#     trythis = medical_qa_model(textinputE)
    q_embedding, a_embedding = tf.unstack(medical_qa_model(textinputE), axis=1)
    return a_embedding.numpy()

In [0]:
askDocspd['A_FFNN_embeds'] = askDocspd.apply(AgetEmbeds, axis=1)

In [0]:
askDocspd['A_FFNN_embeds'].head()

0    [[-0.09869906, 0.20935602, -0.42185766, -0.33665547, 0.48797682, -0.007028032, 0.07065214, -0.16578242, -0.26237178, 0.24004409, -0.22438423, 0.18115889, 0.4567215, -0.02180059, 0.14635685, 0.74769473, -0.026884364, -0.2658844, 0.35665464, 0.3084...
1    [[0.02158143, -0.082602814, -0.17985626, -0.19048187, 0.022836702, -0.09730069, 0.3418817, -0.15769783, -0.10843049, 0.15295486, 0.18546492, 0.09176144, 0.64303726, 0.27884188, 0.041989554, 0.09366697, 0.1290735, -0.11779204, 0.46386075, 0.239391...
2    [[-0.16730975, 0.06444351, 0.009257315, 0.13030894, 0.1986657, 0.111179024, 0.050693758, -0.2810906, -0.16696753, 0.22874801, 0.060951676, 0.024106182, 0.10953758, -0.14491685, 0.23797303, 0.14848688, -0.06242373, -0.3347124, 0.35131043, 0.427723...
3    [[-0.06909141, -0.11909217, -0.2177947, -0.31697032, 0.045378305, 0.26905084, 0.2377099, -0.05351404, -0.14071696, 0.2628327, 0.056118537, -0.07538253, 0.8720115, -0.14583397, 0.07096485, 0.5082699, 0.21771231, -0.33978415, 0.3260

In [0]:
askDocspd.head(20)

,question,answer,question_bert,answer_bert,Q_FFNN_embeds,A_FFNN_embeds
0,Trying to get y friend to see a doctor but she wants to wait and see what happens. I'm worried sick and would like to know hat she might haveMy friend found a dime sized red lump a bit to the right of her clitoris in the shower this morning. its ...,This is most likely a Bartholin's Gland Cyst. On either side of the labia majora is a called a Bartholin's gland. Occasionally the ducts that empty the gland can become blocked causing the gland to develop a cyst. Sometimes they can go away on th...,[[ 5.03139086e-02 1.75226659e-01 -1.97714433e-01 -7.32559934e-02\n 3.20691824e-01 -1.78071290e-01 2.48940829e-02 -3.78650844e-01\n -3.47410381e-01 2.75857508e-01 -2.56084532e-01 2.22180486e-01\n -1.10391743e-01 1.29870668e-01 1.20820425...,[[-9.86990631e-02 2.09356025e-01 -4.21857655e-01 -3.36655468e-01\n 4.87976819e-01 -7.02803209e-03 -3.16477306e-02 -1.65782422e-01\n -2.62371778e-01 2.40044087e-01 -2.24384233e-01 1.41379043e-01\n 1.54396653e-01 -2.18005907e-02 1.46356851...,"[[0.05031391, 0.17522666, 0.71465725, 0.5769979, 0.32069182, -0.17807129, 1.2392216, -0.37865084, 0.1199767, 0.2758575, -0.002445519, 0.3054453, -0.11039174, 0.56973076, 0.120820425, -0.107291855, -0.17624098, 0.23657487, -0.23213816, 0.33558795,...","[[-0.09869906, 0.20935602, -0.42185766, -0.33665547, 0.48797682, -0.007028032, 0.07065214, -0.16578242, -0.26237178, 0.24004409, -0.22438423, 0.18115889, 0.4567215, -0.02180059, 0.14635685, 0.74769473, -0.026884364, -0.2658844, 0.35665464, 0.3084..."
1,"Ease my worries pleaseMy husband is about to see a hematologist and googling around has me worried that he's got something serious, like aplastic anemia. I know we'll find out soon enough, but in the meantime, could you please suggest alternative...","Would be helpful with the exact values of the CBC. And B12, Folate and Ferritin values.\n\nAre liver and kidney function tests okay?\n\nWhat are his ethnic origins?,Is he diabetic. Does he take any other medication?,Does he take aspirin?\nDid he ...",[[-6.82685152e-02 4.29603830e-02 -3.32764566e-01 -6.10484481e-02\n 1.12031415e-01 -2.02944130e-01 -9.14147869e-02 -2.58494318e-01\n -2.18344301e-01 2.87112653e-01 -6.67973831e-02 -9.51428413e-02\n -8.95148739e-02 -1.64768383e-01 2.71721900...,[[ 2.15814300e-02 -8.26028138e-02 -1.79856256e-01 -1.90481871e-01\n 2.28367019e-02 -9.73006934e-02 -1.17688790e-01 -1.57697827e-01\n -1.08430490e-01 1.52954862e-01 1.85464919e-01 -1.62801355e-01\n 2.00268812e-02 -2.12160777e-02 4.19895537...,"[[-0.068268515, 0.042960383, 0.17118096, 0.26986143, 0.112031415, -0.20294413, 0.7771517, -0.25849432, 0.59284496, 0.6635877, 0.3940065, -0.09514284, -0.089514874, 0.33704883, 0.2717219, -0.18564875, 0.1500851, 1.5225036, -0.21757159, 0.2978152, ...","[[0.02158143, -0.082602814, -0.17985626, -0.19048187, 0.022836702, -0.09730069, 0.3418817, -0.15769783, -0.10843049, 0.15295486, 0.18546492, 0.09176144, 0.64303726, 0.27884188, 0.041989554, 0.09366697, 0.1290735, -0.11779204, 0.46386075, 0.239391..."
2,"Severe shoulder, neck and back pain after tetanus vaccinationA little less than a week ago I received my tetanus booster shot (the one you receive every decade, at least here in Australia).\n\nA few days later I began experiencing pain around my ...","I seriously don't think your pain derives from the vaccination.\n\nIt looks like trapezoid contracture (is it the right word in english?).\n\nHeat and posture care should help you, elseways some muscular relaxant could be helpful.\n\nAnyways, at ...",[[-9.59900767e-02 3.72468904e-02 -3.78887504e-01 -4.50596698e-02\n 1.33349165e-01 -4.34329249e-02 4.04662602e-02 -2.44166061e-01\n -1.02068119e-01 4.49610293e-01 -5.45308404e-02 -8.46140832e-02\n -4.96546812e-02 -2.82016784e-01 3.25671524...,[[-1.67309746e-01 6.44435063e-02 9.25731473e-03 1.30308941e-01\n 1.98665693e-01 1.11179024e-01 3.86667326e-02 -2.81090587e-01\n -1.66967526e-01 2.28748009e-01 6.09516762e-02 2.41061822e-02\n 2.800336

In [0]:
del askDocspd['question_bert']
del askDocspd['answer_bert']

In [0]:
askDocspd.head(20)

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,Trying to get y friend to see a doctor but she wants to wait and see what happens. I'm worried sick and would like to know hat she might haveMy friend found a dime sized red lump a bit to the right of her clitoris in the shower this morning. its ...,This is most likely a Bartholin's Gland Cyst. On either side of the labia majora is a called a Bartholin's gland. Occasionally the ducts that empty the gland can become blocked causing the gland to develop a cyst. Sometimes they can go away on th...,"[[0.05031391, 0.17522666, 0.71465725, 0.5769979, 0.32069182, -0.17807129, 1.2392216, -0.37865084, 0.1199767, 0.2758575, -0.002445519, 0.3054453, -0.11039174, 0.56973076, 0.120820425, -0.107291855, -0.17624098, 0.23657487, -0.23213816, 0.33558795,...","[[-0.09869906, 0.20935602, -0.42185766, -0.33665547, 0.48797682, -0.007028032, 0.07065214, -0.16578242, -0.26237178, 0.24004409, -0.22438423, 0.18115889, 0.4567215, -0.02180059, 0.14635685, 0.74769473, -0.026884364, -0.2658844, 0.35665464, 0.3084..."
1,"Ease my worries pleaseMy husband is about to see a hematologist and googling around has me worried that he's got something serious, like aplastic anemia. I know we'll find out soon enough, but in the meantime, could you please suggest alternative...","Would be helpful with the exact values of the CBC. And B12, Folate and Ferritin values.\n\nAre liver and kidney function tests okay?\n\nWhat are his ethnic origins?,Is he diabetic. Does he take any other medication?,Does he take aspirin?\nDid he ...","[[-0.068268515, 0.042960383, 0.17118096, 0.26986143, 0.112031415, -0.20294413, 0.7771517, -0.25849432, 0.59284496, 0.6635877, 0.3940065, -0.09514284, -0.089514874, 0.33704883, 0.2717219, -0.18564875, 0.1500851, 1.5225036, -0.21757159, 0.2978152, ...","[[0.02158143, -0.082602814, -0.17985626, -0.19048187, 0.022836702, -0.09730069, 0.3418817, -0.15769783, -0.10843049, 0.15295486, 0.18546492, 0.09176144, 0.64303726, 0.27884188, 0.041989554, 0.09366697, 0.1290735, -0.11779204, 0.46386075, 0.239391..."
2,"Severe shoulder, neck and back pain after tetanus vaccinationA little less than a week ago I received my tetanus booster shot (the one you receive every decade, at least here in Australia).\n\nA few days later I began experiencing pain around my ...","I seriously don't think your pain derives from the vaccination.\n\nIt looks like trapezoid contracture (is it the right word in english?).\n\nHeat and posture care should help you, elseways some muscular relaxant could be helpful.\n\nAnyways, at ...","[[-0.09599008, 0.03724689, 0.10206425, 0.4240935, 0.13334917, -0.043432925, 1.1233978, -0.24416606, 0.57163686, 0.4496103, -0.05453084, -0.060560953, -0.04965468, -0.28201678, 0.32567152, -0.12495111, -0.051384494, 1.0094256, -0.15498431, 0.43008...","[[-0.16730975, 0.06444351, 0.009257315, 0.13030894, 0.1986657, 0.111179024, 0.050693758, -0.2810906, -0.16696753, 0.22874801, 0.060951676, 0.024106182, 0.10953758, -0.14491685, 0.23797303, 0.14848688, -0.06242373, -0.3347124, 0.35131043, 0.427723..."
3,"Knee problemHere's the run down. This past weekend my wife and I took part in a 3-day walk for breast cancer. It was 60 miles, so clearly a lot of work on my knees. Day 1 went fine, feet hurt, but to be expected. Day two came and I was having...",It's possible that you caused some damage while playing sports and now with a heavy amount of activity caused aggravation of that injury. Its possible to cause a small tear in a meniscus and go years without any problems until a specific event ca...,"[[-0.012918322, -0.020504545, 0.4435106, 0.19794388, 0.076883316, -0.21038833, 0.5508412, -0.16301246, 0.53401446, 0.48718905, 0.5478626, 0.13116859, -0.024213262, 0.0022163023, 0.28197163, -0.17161314, 0.30555886, 0.1917058, -0.40395558, 0.17525...","[[-0.06909141, -0.11909217, -0.2177947, -0.31697032, 0.045378305, 0.26905084, 0.2377099, -0.05351404, -0.14071696, 0.2628327, 0.056118537, -0.07538253, 0.8720115, -0.14583397, 

In [0]:
askDocspd.to_csv('askDocsFFNNEmbeddings.csv')

In [0]:
os.listdir()

In [0]:
# os.remove("webMDFFNNEmbeddings.csv")

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


    100% |████████████████████████████████| 993kB 24.3MB/s 


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# 2. Create & upload a file text file.
uploadModel = drive.CreateFile()
uploaded = drive.CreateFile()
uploadModel.SetContentFile( 'askDocsFFNNEmbeddings.csv')
uploadModel.Upload()

W0427 05:58:06.826507 140313095235456 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNo